In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
landscape = dataset1[0].loc['10μM'].tolist()

In [5]:
greedy_count = 0
total_count = 1000
for i in range(0,total_count):
    results = simulate(landscape, timesteps=1200, seed='0001')
    if results['greedy_path'] == results['actual_path']:
        greedy_count += 1
greedy_count / total_count

0.81

In [4]:
results = simulate(landscape, timesteps=700, seed='0001')
plot_simulation(results)

In [8]:
results['greedy_path']

['0001', '1001', '1011', '1111', '1110']

In [9]:
results['actual_path']

['0001', '1001', '1011', '1111', '1110']